# In-sensor reservoir computing for language learning via two-dimensional memristors
单层感知器、支持向量机和逻辑回归等监督学习模型用于读出层

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

train_data = pd.read_csv("./train_data_01.csv")
test_data = pd.read_csv("./test_data_01.csv")

In [2]:
from sklearn.utils import shuffle

train_data = shuffle(train_data)
test_data = shuffle(test_data)

In [3]:
X_train, y_train = train_data.iloc[:,1:6], train_data.loc[:,"LETTER"]
X_test, y_test = test_data.iloc[:,1:6], test_data.loc[:,"LETTER"]

## Single layer perceptron

In [4]:
import tensorflow as tf

# 创建一个顺序模型
model = tf.keras.models.Sequential([
    # 添加一个全连接层，包含 5 个神经元，激活函数为 softmax，输入形状为 (5,)
    tf.keras.layers.Dense(5, activation='softmax', input_shape=(5,))
])

# 编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),  # 使用 Adam 优化器，学习率为 0.001
              loss='sparse_categorical_crossentropy',  # 使用稀疏类别交叉熵作为损失函数
              metrics=['accuracy'])  # 使用准确率作为评估指标

In [5]:
# 训练模型
model.fit(X_train, y_train, 
          epochs=1000,  # 训练的迭代次数为 1000
          batch_size=2**4,  # 批次大小为 2 的 4 次方，即 16
          validation_data=(X_test, y_test))  # 使用验证数据集 (X_test, y_test) 进行评估

In [6]:
# 评估模型在测试集上的表现，并打印测试得分
# model.evaluate 返回 [损失, 准确率]，此处取 [1] 即准确率
print("Test score: {:.2f}".format(model.evaluate(X_test, y_test)[1]))

## Logistic Regression

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# 创建一个流水线，包含逻辑回归模型
# 逻辑回归使用 elasticnet 惩罚项，使用 saga 求解器，最大迭代次数为 300
pipe = make_pipeline(LogisticRegression(penalty="elasticnet", solver="saga", max_iter=300))

# 定义参数网格，用于超参数调优
# 'logisticregression__C' 表示逻辑回归模型的正则化强度参数的不同取值
# 'logisticregression__l1_ratio' 表示 L1 惩罚项和 L2 惩罚项的比率，取值范围为 0 到 1
param_grid = {
    'logisticregression__C': [0.01, 0.1, 1, 10, 100],
    'logisticregression__l1_ratio': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
}

In [8]:
from sklearn.model_selection import GridSearchCV

# 使用网格搜索进行超参数调优
# 创建 GridSearchCV 对象，传入流水线和参数网格
# cv=5 表示使用 5 折交叉验证，n_jobs=-1 表示使用所有可用 CPU 核心进行并行计算
grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)

# 使用训练数据拟合模型，以找到最佳参数组合
grid.fit(X_train, y_train)

In [9]:
# 打印网格搜索得到的最佳模型
print("Best:\n", grid.best_estimator_)

In [1]:
# 打印训练集上的得分（准确率）
print("Train score: {:.2f}".format(grid.score(X_train, y_train)))

# 打印测试集上的得分（准确率）
print("Test score: {:.2f}".format(grid.score(X_test, y_test)))

In [11]:
# 打印网格搜索期间交叉验证得到的最佳平均得分
print(grid.best_score_)

In [12]:
# 将网格搜索的交叉验证结果转换为 DataFrame
results = pd.DataFrame(grid.cv_results_)

# 转置结果 DataFrame 并显示（为了更方便查看每个参数组合的评估结果）
display(results.T)

## Linear SVM 

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

# 定义超参数网格，用于 SVM 模型的超参数调优
param_grid = [
    # 第一组参数配置，使用线性核函数
    {'kernel': ['linear'],
     'C': [0.01, 0.1, 1, 10]},  # 正则化参数 C 的不同取值

    # 第二组参数配置，使用多项式核函数
    {'kernel': ['poly'],
     'C': [0.01, 0.1, 1, 10],  # 正则化参数 C 的不同取值
     'degree': [2, 3, 4]}  # 多项式核的度数
]

In [14]:
from sklearn.model_selection import GridSearchCV

# 使用网格搜索进行超参数调优
# 创建 GridSearchCV 对象，传入 SVM 模型（SVC()）和参数网格（param_grid）
# cv=5 表示使用 5 折交叉验证，n_jobs=-1 表示使用所有可用 CPU 核心进行并行计算
grid = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)

# 使用训练数据拟合模型，以找到最佳参数组合
grid.fit(X_train, y_train)

In [15]:
# 打印网格搜索得到的最佳模型
print("Best:\n", grid.best_estimator_)

In [16]:
# 打印训练集上的得分（准确率）
print("Train score: {:.2f}".format(grid.score(X_train, y_train)))

# 打印测试集上的得分（准确率）
print("Test score: {:.2f}".format(grid.score(X_test, y_test)))

In [17]:
# 打印网格搜索期间交叉验证得到的最佳平均得分
print(grid.best_score_)

In [18]:
# 将网格搜索的交叉验证结果转换为 DataFrame
results = pd.DataFrame(grid.cv_results_)

# 转置结果 DataFrame 并显示（为了更方便查看每个参数组合的评估结果）
display(results.T)